## Hypothesis Testing

**Problem statement ->**
<br>A F&B manager wants to determine whether there is any significant difference in the diameter of the cutlet between two units.
 A randomly selected sample of cutlets was collected from both units and measured? Analyze the data and draw inferences at
 5% significance level. Please state the assumptions and tests that you carried out to check validity of the assumptions.

So to determine whther the two distribution are significantly different

In [8]:
import pandas as pd
import numpy as np
from scipy import stats

In [5]:
df = pd.read_csv("G:\Github\DS-assignments-python\Hypothesis testing python\Cutlets.csv")


In [6]:
df.describe()

,Unit A,Unit B
count,35.000000,35.000000
mean,7.019091,6.964297
std,0.288408,0.343401
min,6.437600,6.038000
25%,6.831500,6.753600
50%,6.943800,6.939900
75%,7.280550,7.195000
max,7.516900,7.545900


First always check if the two ditributions are normal or not ...<br> external conditions are same... 

In [9]:
stats.shapiro(df["Unit A"])

ShapiroResult(statistic=0.9649458527565002, pvalue=0.3199819028377533)

In [10]:
stats.shapiro(df["Unit A"])

ShapiroResult(statistic=0.9649458527565002, pvalue=0.3199819028377533)

both have pvalue >0.05.. thus not significant.. <br>specifiying that null hypothesis of data being normal cannot be rejected
<br> **thus the distributions are normal**

Now the question is - were the external conditions same....<br>yes they were <br> **So we go for paired T test**

In [11]:
stats.ttest_rel(df["Unit A"],df["Unit B"])

Ttest_relResult(statistic=0.7536787225614314, pvalue=0.4562300768038412)

Pvalue is greater than 0.05 (as 95% confi interval..for 99% ..pvalue greater than 0.01 would be consider)
<br>.. thus saying insignificant and failed to reject null hypothesis
<br> **the Distribution statstically not so different OR failed to reject the thesis that they are different** 

-------------------------------------------

### Data is discrete

Fantaloons Sales managers commented that % of males versus females walking in to the store differ based on day of the week.<br>
 Analyze the data and determine whether there is evidence at 5 % significance level to support this hypothesis.


In [12]:
# here both data is discrete

df2 = pd.read_csv("G:\Github\DS-assignments-python\Hypothesis testing python\Faltoons.csv")

print(df2.describe())

       Weekdays Weekend
count       400     400
unique        2       2
top      Female  Female
freq        287     233


In [14]:
df2.head(5)

,Weekdays,Weekend
0,Male,Female
1,Female,Male
2,Female,Male
3,Male,Female
4,Female,Female


In [15]:
df2["Weekdays"].value_counts()

Female    287
Male      113
Name: Weekdays, dtype: int64

In [16]:
df2["Weekend"].value_counts()

Female    233
Male      167
Name: Weekend, dtype: int64

H0 being the praportion are same 
<br>Ha is praportion are different ...thus Weekend\Weekdays attract more male\female 
<br>So we have to Perform 2 praportion test

In [17]:
len(df2)

400

In [19]:
female_weekdays,Total_visit_weekdays = (287,400)
female_weekends,Total_visit_weekends = (233,400)

In [22]:
female_numbers = np.array([female_weekdays,female_weekends])
total_visits = np.array([Total_visit_weekdays,Total_visit_weekends])

In [24]:
from statsmodels.stats.proportion import proportions_ztest

In [28]:
stat, p_value = proportions_ztest(count=female_numbers, nobs=total_visits,  alternative='two-sided')
p_value

6.26114287794602e-05

p value lower than 0.05 thus we failed to reject null hypothesis
<br>meaning - on weekdays and weekend the praportion are not significantly different enough tos say that the weekend\weekdays affects females checking the shop

---------------------------------------------------------------------------------------------


## data is continuous

Comapny wants to check whether the quality of supply given by suppliers A,B and C are different or same

Ho -> Quality is same <br>
Ha -> Quality is statiscally different.. and somethings needs to be done about this 

In [29]:
cof=pd.read_csv("G:\Github\DS-assignments-python\Hypothesis testing python\ContractRenewal_Data(unstacked).csv")

In [30]:
cof.head(5)

,Supplier A,Supplier B,Supplier C
0,6.15,7.87,7.41
1,6.22,5.21,3.61
2,6.76,7.94,7.23
3,4.29,7.36,5.53
4,7.08,6.17,3.97


In [31]:
cof.columns="SupplierA","SupplierB","SupplierC"

In [32]:
#Normality test 
SupA=stats.shapiro(cof.SupplierA)    #Shapiro Test
SupA_pValue=SupA[1]
print("p-value is: "+str(SupA_pValue))

SupB=stats.shapiro(cof.SupplierB)
SupB_pValue=SupB[1]
print("p-value is: "+str(SupB_pValue))

SupC=stats.shapiro(cof.SupplierC)
SupC_pValue=SupC[1]
print("p-value is: "+str(SupC_pValue))

p-value is: 0.8961844444274902
p-value is: 0.6483432650566101
p-value is: 0.5719023942947388


All are normal so we will now test for variance ...if They were not normal we would have to go with Mood Median Test

In [35]:
#Varience Test 
print(stats.levene(cof.SupplierA, cof.SupplierB))
print(stats.levene(cof.SupplierB, cof.SupplierC))
print(stats.levene(cof.SupplierC, cof.SupplierA))

LeveneResult(statistic=0.03382395609148779, pvalue=0.8542383784793752)
LeveneResult(statistic=0.24333902699144816, pvalue=0.6222596854892893)
LeveneResult(statistic=0.4712013339466693, pvalue=0.49310323292578306)


All have same variance as each have pvalue greater than 0.05.... failed to reject the thesis that varaince are same

Thus we go for **ANOVA Test**...else for diff variances we would go for ANOM test

In [39]:
#One way Anova
#Importing the data set of contractrenewal 
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [41]:
mod=ols('SupplierA~SupplierB+SupplierC',data=cof).fit()
aov_table=anova_lm(mod,type=2)
#In general, if there is no significant interaction effect,
#then type II is more powerful, and follows the principle of marginality.
#If interaction is present, then type II is inappropriate while type III can still be used,
#but results need to be interpreted with caution (in the presence of interactions,
#main effects are rarely interpretable).
print(aov_table)

              df      sum_sq   mean_sq         F    PR(>F)
SupplierB    1.0    0.943234  0.943234  0.519896  0.472324
SupplierC    1.0    3.344433  3.344433  1.843401  0.177166
Residual   117.0  212.269992  1.814273       NaN       NaN


Pvalues are greater than 0.05.. thus failed to rject null hypo
<br>thus theres no significant difference in the quality of all 3 supplier 